In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import zipcodes
from haversine import haversine
import json
os.getcwd()

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [2]:
Nielsen_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1,usecols=[0,2,6])
Nielsen_DMA.columns=['zip_cd','DMA','State']
Nielsen_DMA=Nielsen_DMA.drop_duplicates()

state_dict=Nielsen_DMA[['zip_cd','State']].drop_duplicates()
state_dict=state_dict.groupby(['zip_cd'])['State'].apply(list).to_frame().reset_index()
state_dict=state_dict.set_index('zip_cd').to_dict()['State']

DMA_dict=Nielsen_DMA[['zip_cd','DMA']].drop_duplicates()
DMA_dict=DMA_dict.groupby(['zip_cd'])['DMA'].apply(list).to_frame().reset_index()
DMA_dict=DMA_dict.set_index('zip_cd').to_dict()['DMA']



In [3]:
df_zips_2018=pd.read_csv("/home/jian/SunnyD/Waltmart_TA_20190620/last_year_file/Jay_3_brands_zip_within_10mile.csv",
                           dtype=str)
df_zips_2018['zip_within_10_mile']=df_zips_2018['zip_within_10_mile'].apply(lambda x: eval(x))
df_zips_2018['lat']=df_zips_2018['lat'].astype(float)
df_zips_2018['lng']=df_zips_2018['lng'].astype(float)

df_zips_2018_walmart=df_zips_2018[df_zips_2018['store_id'].apply(lambda x: x.split("_")[0]=="walmart")]

In [4]:
df_zips_2018_walmart.head(2)

,store_id,zip_code,lat,lng,zip_within_10_mile,number_of_dma,dma,dma2,dma3
0,walmart_2071,99515,61.140228,-149.869969,"[99519, 99518, 99515, 99514, 99517, 99516, 995...",1,ANCHORAGE,NaN,NaN
1,walmart_2188,99577,61.309072,-149.535530,"[99577, 99505, 99521, 99567]",1,ANCHORAGE,NaN,NaN


In [5]:
df_output=pd.DataFrame(columns=['store_id','zip_code_10_miles'])
for ind,row in df_zips_2018_walmart.iterrows():
    store_id=row['store_id']
    zip_list=row['zip_within_10_mile']
    df=pd.DataFrame({"store_id":[store_id]*len(zip_list),"zip_code_10_miles":zip_list})
    df_output=df_output.append(df)

In [6]:
def store_city(x):
    try:
        y=zipcodes.matching(x)[0]['city']
    except:
        y=np.nan
    return y

def store_state(x):
    try:
        y=state_dict[x]
    except:
        try:
            y=zipcodes.matching(x)[0]['state']
        except:
            y=np.nan
    return y

def store_DMA(x):
    try:
        y=DMA_dict[x]
    except:
        y=np.nan
    return y

In [7]:
store_info=df_zips_2018_walmart[['store_id','zip_code','lat','lng']]

store_info=store_info.rename(columns={"zip_code":"store_zip",'lat':"store_lat",'lng':"store_lng"})
store_info['store_DMA']=store_info['store_zip'].apply(lambda x: store_DMA(x))
store_info['store_city']=store_info['store_zip'].apply(lambda x: store_city(x))
store_info['store_state']=store_info['store_zip'].apply(lambda x: store_state(x))

store_info.head(2)

,store_id,store_zip,store_lat,store_lng,store_DMA,store_city,store_state
0,walmart_2071,99515,61.140228,-149.869969,[ANCHORAGE],ANCHORAGE,[AK]
1,walmart_2188,99577,61.309072,-149.535530,[ANCHORAGE],EAGLE RIVER,[AK]


In [8]:
df_output_by_store_zip=pd.merge(store_info,df_output,on="store_id",how="left")
df_output_by_store_zip['target_zip_DMA']=df_output_by_store_zip['zip_code_10_miles'].apply(lambda x: store_DMA(x))
df_output_by_store_zip['target_zip_city']=df_output_by_store_zip['zip_code_10_miles'].apply(lambda x: store_city(x))
df_output_by_store_zip['target_zip_state']=df_output_by_store_zip['zip_code_10_miles'].apply(lambda x: store_state(x))

df_output_by_store_zip['distance_miles']=np.nan

In [9]:
df_output_by_store_zip=df_output_by_store_zip.reset_index()
del df_output_by_store_zip['index']

for ind,row in df_output_by_store_zip.iterrows():
    store_center=[row['store_lat'],row['store_lng']]
    zip_cd=row['zip_code_10_miles']
    zip_center=zip_centers[zip_cd]
    dist=haversine(store_center,zip_center,miles=True)
    df_output_by_store_zip.loc[ind,'distance_miles']=dist

In [10]:
df_output_by_store_zip.head(2)

,store_id,store_zip,store_lat,store_lng,store_DMA,store_city,store_state,zip_code_10_miles,target_zip_DMA,target_zip_city,target_zip_state,distance_miles
0,walmart_2071,99515,61.140228,-149.869969,[ANCHORAGE],ANCHORAGE,[AK],99519,[ANCHORAGE],ANCHORAGE,[AK],4.896374
1,walmart_2071,99515,61.140228,-149.869969,[ANCHORAGE],ANCHORAGE,[AK],99518,[ANCHORAGE],ANCHORAGE,[AK],1.341670


In [11]:
df_output_zip=df_output_by_store_zip[['zip_code_10_miles','target_zip_DMA','target_zip_city','target_zip_state','store_id','distance_miles','store_DMA','store_city','store_state']]

df_output_zip=df_output_zip.rename(columns={"store_id":"nearest_store"})

df_output_zip=df_output_zip.sort_values(["zip_code_10_miles",'distance_miles','nearest_store'],ascending=[True,True,True]).drop_duplicates("zip_code_10_miles")
df_output_zip.shape

(16877, 9)

In [18]:
df_merge_min_dist=df_output_zip[['zip_code_10_miles','nearest_store','distance_miles']]
df_merge_min_dist=df_merge_min_dist.rename(columns={"distance_miles":"min_dist_to_nearest_store"})

df_output_by_store_zip=pd.merge(df_output_by_store_zip,df_merge_min_dist,on="zip_code_10_miles",how="left")

In [20]:
os.getcwd()

'/home/jian/SunnyD/Waltmart_TA_20190620'

In [21]:
writer=pd.ExcelWriter("./SunnyD_zips_in_10_miles_of_Walmart_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_zip.to_excel(writer,"unique_zips",index=False)
df_output_by_store_zip.to_excel(writer,"by_store_zip",index=False)
writer.save()

In [22]:
df_output_by_store_zip['store_id'].nunique()

2596